# Eikon Data API with SCREENER example - JupyterLab

This Notebook demonstrates how to use Eikon SCREENER application to find the companies with the [Eikon Data API](https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api).

In [1]:
import refinitiv.dataplatform.eikon as ek
from dotenv import load_dotenv
import os

%load_ext dotenv
%dotenv

### Connecting to Eikon Data API and RDP Session

This code sets 
- the `app_id` to connect to the **Eikon Data API** via the **Refinitiv Workspace/Eikon Desktop Application** which needs to be running locally. 
- the RDP `app_key`, `usrname`, and `password` to connect to the RDP Open Platform Session.

You need the following Environment Variables in your OS setting:

```
#RDP
RDP_USERNAME=YOUR_RDP_USERNAME
RDP_PASSWORD=YOUR_RDP_PASSWORD
RDP_APP_KEY=YOUR_RDP_APP_KEY_HERE
# Eikon Data API
EIKON_APP_KEY=YOUR_EIKON_APP_ID_HERE
```

Alternatively, you can create a text file with **filename** `.env` having the above contents:
    
This file (or OS Environment Variables) should be readily available (e.g. in the current working directory) for the next steps.

Initialize Eikon Data API session.

Please note that the Refinitiv Workspace/Eikon application integrates a Data API proxy that acts as an interface between the Eikon Data API Python library and the Eikon Data Platform. For this reason, the Refinitiv Workspace/Eikon application must be running when you use the Eikon Data API Python library.

In [2]:
ek.set_app_key(os.getenv('EIKON_APP_KEY'))

## SCREENER to Python

Article: [Find Your Right Companies with SCREENER | Eikon Data APIs(Python)](https://developers.refinitiv.com/en/article-catalog/article/find-your-right-companies-with-screener-eikon-data-apis-python).

### Screener Formula

**Excel Formula**: Contain both formula and fields
```
=@TR("SCREEN(U(IN(Equity(active,public,primary))/*UNV:Public*/), IN(TR.HQCountryCode,""CN""), TR.TRESGCScore(Period=FY0)>=50, CURN=USD)","TR.CommonName;TR.HeadquartersCountry;TR.TRESGCScore(Period=FY0)","curn=USD RH=In CH=Fd")
```

Which can be separated to Formula and Fields:

*Formula*
```
"SCREEN(U(IN(Equity(active,public,primary))/*UNV:Public*/), IN(TR.HQCountryCode,""CN""), TR.TRESGCScore(Period=FY0)>=50, CURN=USD)"
```
*Fields*
```
"TR.CommonName;TR.HeadquartersCountry;TR.TRESGCScore(Period=FY0)"
```

Change to **Python Syntax**: 
```
syntax = 'SCREEN(U(IN(Equity(active,public,primary))), IN(TR.HQCountryCode,"CN"), TR.TRESGCScore>=50, CURN=USD)'
```
And **Python field**: 
```
fields = ['TR.CommonName', 'TR.HeadquartersCountry','TR.TRESGCScore']
```

In [3]:
syntax = 'SCREEN(U(IN(Equity(active,public,primary))), IN(TR.HQCountryCode,"CN"), TR.TRESGCScore>=50, CURN=USD)'

fields = ['TR.CommonName', 'TR.HeadquartersCountry','TR.TRESGCScore']

In [4]:
df, err = ek.get_data(syntax, fields)
df

,Instrument,Company Common Name,Country of Headquarters,ESG Combined Score
0,600690.SS,Haier Smart Home Co Ltd,China,54.242808
1,601939.SS,China Construction Bank Corp,China,66.178801
2,0425.HK,Minth Group Ltd,China,55.165821
3,601988.SS,Bank of China Ltd,China,57.658573
4,0867.HK,China Medical System Holdings Ltd,China,64.730653
...,...,...,...,...
207,1951.HK,Jinxin Fertility Group Ltd,China,55.031206
208,9922.HK,Jiumaojiu International Holdings Ltd,China,60.838968
209,6186.HK,China Feihe Ltd,China,51.094033
210,1209.HK,China Resources Mixc Lifestyle Services Ltd,China,51.151221


Or you can add more fields data to that you are interesting.

In [5]:
# TR.TRESGScore = ESGScore, TR.TRESGCControversiesScore = ESG Controversies Score, TR.TRESGCScoreGrade = ESG Combined Score Grade
fields = ['TR.CommonName', 'TR.HeadquartersCountry','TR.TRESGCScore', 'TR.TRESGScore', 'TR.TRESGCControversiesScore', 'TR.TRESGCScoreGrade']

In [6]:
df, err = ek.get_data(syntax, fields)
df

,Instrument,Company Common Name,Country of Headquarters,ESG Combined Score,ESG Score,ESG Controversies Score,ESG Combined Score Grade
0,600690.SS,Haier Smart Home Co Ltd,China,54.242808,54.242808,100.0,B-
1,601939.SS,China Construction Bank Corp,China,66.178801,66.178801,98.333333,B
2,0425.HK,Minth Group Ltd,China,55.165821,55.165821,100.0,B-
3,601988.SS,Bank of China Ltd,China,57.658573,57.658573,68.75,B-
4,0867.HK,China Medical System Holdings Ltd,China,64.730653,64.730653,100.0,B
...,...,...,...,...,...,...,...
207,1951.HK,Jinxin Fertility Group Ltd,China,55.031206,55.031206,100.0,B-
208,9922.HK,Jiumaojiu International Holdings Ltd,China,60.838968,60.838968,100.0,B
209,6186.HK,China Feihe Ltd,China,51.094033,51.094033,100.0,B-
210,1209.HK,China Resources Mixc Lifestyle Services Ltd,China,51.151221,51.151221,100.0,B-


### With Combined Score Grade A

**Excel Formula**
```
=@TR("SCREEN(U(IN(Equity(active,public,primary))/*UNV:Public*/), IN(TR.HQCountryCode,""CN""), TR.TRESGCScore(Period=FY0)>=50, Contains(TR.TRESGCScoreGrade(Period=FY0),""A""), CURN=USD)","TR.CommonName;TR.HeadquartersCountry;TR.TRESGCScore(Pe"&"riod=FY0);TR.TRESGCScoreGrade(Period=FY0)","curn=USD RH=In CH=Fd")
```

**Python Syntax**
```
syntax = 'SCREEN(U(IN(Equity(active,public,primary))), IN(TR.HQCountryCode,"CN"), TR.TRESGCScore>=50, Contains(TR.TRESGCScoreGrade,"A"), CURN=USD)'
```

In [7]:
syntax = 'SCREEN(U(IN(Equity(active,public,primary))), IN(TR.HQCountryCode,"CN"), TR.TRESGCScore>=50, Contains(TR.TRESGCScoreGrade,"A"), CURN=USD)'

In [8]:
df, err = ek.get_data(syntax, fields)
df

,Instrument,Company Common Name,Country of Headquarters,ESG Combined Score,ESG Score,ESG Controversies Score,ESG Combined Score Grade
0,2382.HK,Sunny Optical Technology Group Co Ltd,China,76.888594,76.888594,100,A-
1,2007.HK,Country Garden Holdings Co Ltd,China,81.261232,81.261232,100,A-
2,601628.SS,China Life Insurance Co Ltd,China,78.458262,78.458262,100,A-
3,601919.SS,COSCO Shipping Holdings Co Ltd,China,82.077287,82.077287,100,A-
4,2688.HK,ENN Energy Holdings Ltd,China,75.642564,75.642564,100,A-
5,2018.HK,AAC Technologies Holdings Inc,China,78.354849,78.354849,100,A-
6,NOAH.N,Noah Holdings Ltd,China,77.996627,77.996627,100,A-
7,601012.SS,LONGi Green Energy Technology Co Ltd,China,76.864813,76.864813,100,A-
8,688235.SS,Beigene Ltd,China,77.840992,77.840992,100,A-
9,YUMC.N,Yum China Holdings Inc,China,86.251739,86.251739,100,A


### With Combined Score Grade A or B

**Excel Formula**
```
=@TR("SCREEN(U(IN(Equity(active,public,primary))/*UNV:Public*/), IN(TR.HQCountryCode,""CN""), TR.TRESGCScore(Period=FY0)>=90, Contains(TR.TRESGCScoreGrade(Period=FY0),""A"") OR Contains(TR.TRESGCScoreGrade(Period=FY0),""B""), CURN=USD)","TR."&"CommonName;TR.HeadquartersCountry;TR.TRESGCScore(Period=FY0);TR.TRESGCScoreGrade(Period=FY0)","curn=USD RH=In CH=Fd")
```

**Python Syntax**
```
syntax ='SCREEN(U(IN(Equity(active,public,primary))), IN(TR.HQCountryCode,"CN"), TR.TRESGCScore>=90, Contains(TR.TRESGCScoreGrade,"A") OR Contains(TR.TRESGCScoreGrade,"B"), CURN=USD)'
```

In [9]:
syntax ='SCREEN(U(IN(Equity(active,public,primary))), IN(TR.HQCountryCode,"CN"), '\
    'TR.TRESGCScore>=50,'\
    'Contains(TR.TRESGCScoreGrade,"A") OR Contains(TR.TRESGCScoreGrade,"B"),'\
    'CURN=USD)'

In [10]:
df, err = ek.get_data(syntax, fields)
df

,Instrument,Company Common Name,Country of Headquarters,ESG Combined Score,ESG Score,ESG Controversies Score,ESG Combined Score Grade
0,600690.SS,Haier Smart Home Co Ltd,China,54.242808,54.242808,100.0,B-
1,601939.SS,China Construction Bank Corp,China,66.178801,66.178801,98.333333,B
2,0425.HK,Minth Group Ltd,China,55.165821,55.165821,100.0,B-
3,601988.SS,Bank of China Ltd,China,57.658573,57.658573,68.75,B-
4,0867.HK,China Medical System Holdings Ltd,China,64.730653,64.730653,100.0,B
...,...,...,...,...,...,...,...
207,1951.HK,Jinxin Fertility Group Ltd,China,55.031206,55.031206,100.0,B-
208,9922.HK,Jiumaojiu International Holdings Ltd,China,60.838968,60.838968,100.0,B
209,6186.HK,China Feihe Ltd,China,51.094033,51.094033,100.0,B-
210,1209.HK,China Resources Mixc Lifestyle Services Ltd,China,51.151221,51.151221,100.0,B-
